<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With Python**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

See also `12_math_basics.ipynb`.

## Imports & Configurations

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
import math
import numpy as np
from numpy.random import default_rng

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

## Financial Stochastic Processes

Applying the Monte Carlo method to simulate typical financial processes.

### Geometric Brownian Motion

Assume the following parameters for the Black-Scholes-Merton (1973) stock price model (geometric Brownian motion) as follows:

* $S_0 = 100$, initial price
* $\sigma = 0.2$, volatility (percent in digits)
* $r = 0.06$, constant short rate
* $T = 1$, time horizon (in year fractions)
* $S_T$, future price (log-normally distributed)

It holds $\frac{dS_t}{S_t} = r dt + \sigma dZ_t$, with $Z_t$ a standard brownian motion (random walk).

In discrete time, it holds $S_{t}= S_{t - \Delta t} e^{\left(r - \frac{\sigma^2}{2}\right) \Delta t + \sigma \sqrt{\Delta t} z_t}$, for $t>0, \Delta t > 0$.

See also https://en.wikipedia.org/wiki/Geometric_Brownian_motion.

In [ ]:
rng = default_rng()

In [ ]:
S0, sigma, r, T = 100, 0.2, 0.06, 1.0

In [ ]:
M = 50

In [ ]:
dt = T / M
dt

In [ ]:
rn = (r - sigma ** 2 / 2) * dt + sigma * math.sqrt(dt) * rng.standard_normal(M + 1)

In [ ]:
rn[0] = 0

In [ ]:
gbm = S0 * np.exp(rn.cumsum())

In [ ]:
plt.plot(gbm[:]);

In [ ]:
def simulate_gbm(M, I, T, r, sigma):
    dt = T / M
    rn = (r - sigma ** 2 / 2) * dt + sigma * math.sqrt(dt) * rng.standard_normal((M + 1, I))
    rn[0] = 0
    gbm = S0 * np.exp(rn.cumsum(axis=0))
    return gbm

In [ ]:
I = 1000

In [ ]:
gbm_procs_1 = simulate_gbm(M, I, 1, 0.1, 0.2)

In [ ]:
plt.plot(gbm_procs_1[:, :50], 'b--', lw=1);

In [ ]:
gbm_procs_2 = simulate_gbm(M, I, 1, 0.2, 0.4)

In [ ]:
plt.plot(gbm_procs_2[:, :50], 'r--', lw=1)
plt.plot(gbm_procs_1[:, :50], 'b--', lw=1);

In [ ]:
S1_mean = gbm_procs_1.mean(axis=1)

In [ ]:
S1_mean

In [ ]:
S2_mean = gbm_procs_2.mean(axis=1)

In [ ]:
plt.plot(gbm_procs_2[:, :25], 'r--', lw=1, alpha=0.5)
plt.plot(gbm_procs_1[:, :25], 'b--', lw=1, alpha=0.5)
plt.plot(S2_mean, 'r-', lw=2, label='high drift/vol mean')
plt.plot(S1_mean, 'b-', lw=2, label='low drift/vol mean')
plt.legend();

In [ ]:
ST_1 = gbm_procs_1[-1]

In [ ]:
ST_2 = gbm_procs_2[-1]

In [ ]:
ST_1.mean()

In [ ]:
S0 * math.exp(0.1 * T)

In [ ]:
lr_1 = np.log(ST_1 / S0)

In [ ]:
lr_1.std()  # volatility

In [ ]:
ST_2.mean()

In [ ]:
S0 * math.exp(0.2 * T)

In [ ]:
lr_2 = np.log(ST_2 / S0)

In [ ]:
lr_2.std()  # volatility

In [ ]:
plt.hist(ST_2, bins=30, label='high drift/vol')
plt.hist(ST_1, bins=30, label='low drift/vol')
plt.axvline(ST_2.mean(), color='r', label='high drift/vol mean')
plt.axvline(ST_1.mean(), color='m', label='low drift/vol mean')
plt.legend();

### Jump Diffusion

Assume the Merton (1976) model which adds a jump component to the Black-Scholes-Merton (1973) model:

$ dS_t = (r-r_j)S_t dt + \sigma S_t dZ_t + J_t S_t dN_t$

The meaning of the variables are:

* $S_t$ index level at date $t$
* $r$ constant riskless short rate
* $r_{J}\equiv \lambda \cdot \left(e^{\mu_{J}+\delta^{2}/2}-1\right)$ drift correction for jump to maintain risk neutrality
* $\sigma$ constant volatility of $S$
* $Z_t$ standard Brownian motion
* $J_t$ jump at date $t$ with distribution ...
  * ... $\log(1+J_{t})\approx \mathbf{N}\left(\log(1+\mu_{J})-\frac{\delta^{2}}{2},\delta^{2}\right)$ with ...
  * ... **N** as the cumulative distribution function of a standard normal random variable
* $N_t$ Poisson process with intensity $\lambda$

An Euler discretization scheme is given by:

$S_{t}=S_{t-\Delta t}\left(e^{(r-r_{J}-\sigma^{2}/2)\Delta t+\sigma \sqrt{\Delta t}z^{1}_{t}}+ \left(e^{\mu_{J}+\delta z^{2}_{t}}-1\right)y_{t}\right)$

with $z^1_t, z^2_t$ being standard normally distributed and $y_t$ being Poisson distributed.

See also https://en.wikipedia.org/wiki/Jump_diffusion.

In [ ]:
muj, delta, lamb = -0.5, 0.1, 0.5

In [ ]:
rj = lamb * (math.exp(muj + delta ** 2 / 2) - 1)

In [ ]:
rj

In [ ]:
M = 100

In [ ]:
T

In [ ]:
dt = T / M

In [ ]:
dt

In [ ]:
rng = default_rng(1000)

In [ ]:
rn_diff = ((r - rj - sigma ** 2 / 2) * dt +
           sigma * math.sqrt(dt) * rng.standard_normal(M + 1))

In [ ]:
rn_jump = ((np.exp(muj + delta * rng.standard_normal(M + 1)) - 1) *
           rng.poisson(lamb * dt, M + 1))

In [ ]:
rn = rn_diff + rn_jump

In [ ]:
rn[0] = 0

In [ ]:
rn[:6]

In [ ]:
jd_proc = S0 * np.exp(rn.cumsum())

In [ ]:
plt.plot(jd_proc);

In [ ]:
def simulate_jd(M, I):
    dt = T / M
    rn_diff = ((r - rj - sigma ** 2 / 2) * dt +
           sigma * math.sqrt(dt) * rng.standard_normal((M + 1, I)))
    rn_jump = ((np.exp(muj + delta * rng.standard_normal((M + 1, I))) - 1) *
           rng.poisson(lamb * dt, (M + 1, I)))
    rn = rn_diff + rn_jump
    rn[0] = 0
    jd_procs = S0 * np.exp(rn.cumsum(axis=0))
    return jd_procs

In [ ]:
I = 100000

In [ ]:
jd_procs = simulate_jd(M, I)

In [ ]:
plt.plot(jd_procs[:, :25], 'b--', lw=1);

In [ ]:
ST = jd_procs[-1]

In [ ]:
ST.mean()

In [ ]:
lr = np.log(ST / S0)

In [ ]:
sigma

In [ ]:
lr.std()

In [ ]:
r

In [ ]:
math.log(ST.mean() / S0)

In [ ]:
(np.exp(lr) - 1).mean()  # mean of simple returns

In [ ]:
plt.hist(ST, bins=50);

In [ ]:
plt.hist(lr, bins=50);  # log returns

In [ ]:
plt.hist(np.exp(lr) - 1, bins=50);  # simple returns

## Europen Option Valuation

Assume the previous stock price model and assume that a European option has a strike price of $K = 100$.

The payoff of a European options is:
* $\max(S - K, 0)$ for a European call option
* $\max(K - S, 0)$ for a European put option

In [ ]:
K = 100

* **stock price**: random phenomenon = nature/markets --> stock price (model)
* **option payoff**: random phenomenon = stock price --> option payoff (model)

#### European Call option

In [ ]:
CT = np.maximum(ST - K, 0)

In [ ]:
CT[:10]

In [ ]:
# Monte Carlo Estimator
C0 = math.exp(-r * T) * CT.mean()  # present value

In [ ]:
C0

In [ ]:
CT.mean()  # future value (estimator)

In [ ]:
RC = np.log(CT / C0 + 0.00001)  # log returns

In [ ]:
RC.var()

In [ ]:
RC.std()

In [ ]:
plt.title('European call option payoff')
plt.hist(CT, bins=50, label='with jumps')
plt.legend();

#### European Put option

In [ ]:
PT = np.maximum(K - ST, 0)

In [ ]:
PT[:10]

In [ ]:
# Monte Carlo Estimator
P0 = math.exp(-r * T) * PT.mean()  # present value

In [ ]:
P0

In [ ]:
PT.mean()  # future value (estimator)

In [ ]:
RP = np.log(PT / P0 + 0.00001)  # log returns

In [ ]:
RP.var()

In [ ]:
RP.std()

In [ ]:
plt.title('European put option payoff')
plt.hist(PT, bins=50, label='with jumps')
plt.legend();

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>